There are many ways to implement the model.

In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras

np.random.seed(42)
tf.random.set_seed(42)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
tf.__version__, keras.__version__

('2.0.0', '2.2.4-tf')

In [3]:
# Load the MNIST data
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train / 255.
x_test = x_test / 255.

x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

# Way 1: Sequential with `add` method

In [4]:
model = tf.keras.Sequential()  # Linear stack of layers.
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))  # Here!
model.add(tf.keras.layers.Dense(300, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.fit(x_train, y_train,
          epochs=2,
          batch_size=64,
          validation_data=(x_test, y_test),
          verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 93us/sample - loss: 0.4823 - acc: 0.8281 - val_loss: 0.4003 - val_acc: 0.8561
Epoch 2/2
60000/60000 [==============================] - 5s 78us/sample - loss: 0.3586 - acc: 0.8681 - val_loss: 0.4140 - val_acc: 0.8526


# Way 2: Suquential with `layers` parameter)

In [6]:
# Destroys the current TF graph and creates a new one.
tf.keras.backend.clear_session()

model = tf.keras.Sequential([  # Here!
    # Flattens the input. Does not affect the batch size.
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.fit(x_train, y_train,
          epochs=2,
          batch_size=64,
          validation_data=(x_test, y_test),
          verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 5s 88us/sample - loss: 0.4859 - acc: 0.8251 - val_loss: 0.4034 - val_acc: 0.8567
Epoch 2/2
60000/60000 [==============================] - 5s 89us/sample - loss: 0.3574 - acc: 0.8684 - val_loss: 0.4230 - val_acc: 0.8467


# Way 3: Suquential with `layers` parameter and `InputLayer`)

In [8]:
# Destroys the current TF graph and creates a new one.
tf.keras.backend.clear_session()

model = tf.keras.Sequential([
    # shape tuple (not including the batch axis)
    tf.keras.layers.InputLayer(input_shape=(28, 28)),  # Here!
    # Flattens the input. Does not affect the batch size.
    tf.keras.layers.Flatten(),                         # Here!
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.fit(x_train, y_train,
          epochs=2,
          batch_size=64,
          validation_data=(x_test, y_test),
          verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 98us/sample - loss: 0.4849 - acc: 0.8260 - val_loss: 0.4015 - val_acc: 0.8572
Epoch 2/2
60000/60000 [==============================] - 5s 87us/sample - loss: 0.3577 - acc: 0.8685 - val_loss: 0.4237 - val_acc: 0.8462


**It is generally recommend to use the functional layer API via `tf.keras.Input`, (which creates an `InputLayer`) without directly using `tf.keras.layers.InputLayer`.**

# Way 4: Functional API (`tf.keras.Input`)

When you start from **`tf.keras.Model`**, you should **chain layer calls to specify the model's forward pass, and finally you create your model from inputs and outputs**:

In [10]:
# Destroys the current TF graph and creates a new one.
tf.keras.backend.clear_session()

inputs = tf.keras.Input(shape=(28 ,28))  # used to instantiate a Keras tensor.
flatten = tf.keras.layers.Flatten()(inputs)
dense_1 = tf.keras.layers.Dense(300, activation='relu')(flatten)
dense_2 = tf.keras.layers.Dense(100, activation='relu')(dense_1)
dense_3 = tf.keras.layers.Dense(10, activation='softmax')(dense_2)
model = tf.keras.Model(inputs=inputs, outputs=dense_3)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.fit(x_train, y_train,
          epochs=2,
          batch_size=64,
          validation_data=(x_test, y_test),
          verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 96us/sample - loss: 0.4877 - acc: 0.8260 - val_loss: 0.4307 - val_acc: 0.8460
Epoch 2/2
60000/60000 [==============================] - 5s 84us/sample - loss: 0.3617 - acc: 0.8684 - val_loss: 0.4173 - val_acc: 0.8471


# Way 5: Subclass the `tf.keras.Model` class

**By subclassing the `tf.keras.Model` class: in that case, you should define your layers in `__init__` and you should implement the model's forward pass in `call`.**

In [12]:
# Destroys the current TF graph and creates a new one.
tf.keras.backend.clear_session()


class MyModel(tf.keras.Model):

    def __init__(self):
        """Define layers."""
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense_1 = tf.keras.layers.Dense(300, activation=tf.nn.relu)
        self.dense_2 = tf.keras.layers.Dense(100, activation=tf.nn.relu)
        self.dense_3 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

    def call(self, inputs):
        """Implement the model's forward pass."""
        flatten = self.flatten(inputs)
        dense_1 = self.dense_1(flatten)
        dense_2 = self.dense_2(dense_1)
        outputs = self.dense_3(dense_2)
        return outputs

In [13]:
model = MyModel()

In [14]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [16]:
model.fit(x_train, y_train,
          epochs=2,
          batch_size=64,
          validation_data=(x_test, y_test),
          verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 95us/sample - loss: 0.4864 - acc: 0.8267 - val_loss: 0.4266 - val_acc: 0.8443
Epoch 2/2
60000/60000 [==============================] - 5s 76us/sample - loss: 0.3598 - acc: 0.8684 - val_loss: 0.4243 - val_acc: 0.8454


In [17]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  235500    
_________________________________________________________________
dense_1 (Dense)              multiple                  30100     
_________________________________________________________________
dense_2 (Dense)              multiple                  1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________
